### Import Libraries

In [9]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import month,year,col,sum
from snowflake.snowpark.version import VERSION
from snowflake.core import Root
from snowflake.core.task import Task, StoredProcedureCall
from snowflake.core.task.dagv1 import DAG, DAGTask, DAGOperation
from snowflake.core import CreateMode

# Misc
from datetime import timedelta
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

### Establish Secure Connection to Snowflake

Using the Snowpark Python API, it’s quick and easy to establish a secure connection between Snowflake and Notebook.

 *Note: Other connection options include Username/Password, MFA, OAuth, Okta, SSO*

TIP: Learn more about [Session](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/session) object.

In [32]:
# # Create Snowflake Session object
import yaml
import os

# Load the YAML file
with open('snowpark_connection.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access the connection parameters
connection_parameters = config['connection_parameters']


# Load the YAML file
with open('connection.yaml', 'r') as file:
    config = yaml.safe_load(file)

for key, value in connection_parameters.items():
    if isinstance(value, str) and value.startswith('{{') and value.endswith('}}'):
        env_var_name = value.strip('{} ')
        connection_parameters[key] = os.environ.get(env_var_name, f"Missing ENV: {env_var_name}")

In [30]:
# connection_parameters2 = {
#     "account": "uhb88781",
#     "user": "vhol_streaming1",
#     "role": "VHOL_CDC_AGENT",
#     "warehouse": "compute_wh", 
#     "database": "VHOL_ENG_CDC",
#     "schema": "ENG",
#     "private_key": open('/Users/hajung/Desktop/CDCSimulatorApp/rsa_key.p8', 'r').read()
# }

# s2 = Session.builder.configs(connection_parameters2).create()

In [33]:

session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : HAEBICHAN
Role                        : "ACCOUNTADMIN"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L"
Snowflake version           : 9.9.2
Snowpark for Python version : 1.18.0


### Load Aggregated Campaign Spend Data from Snowflake table into Snowpark DataFrame

Let's first load the campaign spend data. This table contains ad click data that has been aggregated to show daily spend across digital ad channels including search engines, social media, email and video.

*Note: Some other ways to load data in a Snowpark DataFrame*
* *session.sql("select col1, col2... from tableName")*
* *session.read.options({"field_delimiter": ",", "skip_header": 1}).schema(user_schema).csv("@mystage/testCSV.csv")*
* *session.read.parquet("@stageName/path/to/file")*
* *session.create_dataframe([1,2,3], schema=["col1"])*

TIP: Learn more about [Snowpark DataFrames](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/dataframe).


In [34]:
session.sql('select current_warehouse()').show()

-------------------------
|"CURRENT_WAREHOUSE()"  |
-------------------------
|DASH_L                 |
-------------------------



In [35]:
session.sql('use warehouse compute_wh').show()

------------------------------------
|"status"                          |
------------------------------------
|Statement executed successfully.  |
------------------------------------



In [36]:
session.sql('select current_warehouse()').show()

-------------------------
|"CURRENT_WAREHOUSE()"  |
-------------------------
|COMPUTE_WH             |
-------------------------



In [37]:
snow_df_spend = session.table('CAMPAIGN_SPEND')

Actions like *show(), collect(), count()* send the DataFrame SQL for execution on the server

*Note: History object provides the query ID which can be helpful for debugging as well as the SQL query executed on the server.*

In [38]:
with session.query_history() as history:
    snow_df_spend.show(20)
history.queries

------------------------------------------------------------------------------------------------------
|"CAMPAIGN"              |"CHANNEL"      |"DATE"      |"TOTAL_CLICKS"  |"TOTAL_COST"  |"ADS_SERVED"  |
------------------------------------------------------------------------------------------------------
|winter_sports           |video          |2012-06-03  |213             |1762          |426           |
|sports_across_cultures  |video          |2012-06-02  |87              |678           |157           |
|building_community      |search_engine  |2012-06-03  |66              |471           |134           |
|world_series            |social_media   |2017-12-28  |72              |591           |149           |
|winter_sports           |email          |2018-02-09  |252             |1841          |473           |
|spring_break            |video          |2017-11-14  |162             |1155          |304           |
|nba_finals              |email          |2017-11-22  |68              |4

[QueryRecord(query_id='01bbc042-0305-01ef-0057-0a87047252ce', sql_text='SELECT  *  FROM CAMPAIGN_SPEND LIMIT 20')]

### Total Spend per Year and Month For All Channels

Let's transform the data so we can see total cost per year/month per channel using _group_by()_ and _agg()_ Snowpark DataFrame functions.

TIP: For a full list of functions, refer to the [documentation](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/functions).

In [39]:
# Stats per Month per Channel
snow_df_spend_per_channel = snow_df_spend.group_by(year('DATE'), month('DATE'),'CHANNEL').agg(sum('TOTAL_COST').as_('TOTAL_COST')).\
    with_column_renamed('"YEAR(DATE)"',"YEAR").with_column_renamed('"MONTH(DATE)"',"MONTH").sort('YEAR','MONTH')

snow_df_spend_per_channel.show(10)

---------------------------------------------------
|"YEAR"  |"MONTH"  |"CHANNEL"      |"TOTAL_COST"  |
---------------------------------------------------
|2012    |5        |search_engine  |516431        |
|2012    |5        |video          |516729        |
|2012    |5        |email          |517208        |
|2012    |5        |social_media   |517618        |
|2012    |6        |video          |501098        |
|2012    |6        |search_engine  |506497        |
|2012    |6        |social_media   |504679        |
|2012    |6        |email          |501947        |
|2012    |7        |search_engine  |522780        |
|2012    |7        |email          |518405        |
---------------------------------------------------



### Pivot on Channel: Total Spend Across All Channels

 Let's further transform the campaign spend data so that **each row will represent total cost across all channels** per year/month using _pivot()_ and _sum()_ Snowpark DataFrame functions. This transformation will enable us to join with the revenue table such that we will have our input features and target variable in a single table for model training. 

 TIP: For a full list of functions, refer to the [documentation](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/functions).

In [42]:
snow_df_spend_per_month = snow_df_spend_per_channel.pivot('CHANNEL',['search_engine','social_media','video','email']).sum('TOTAL_COST').sort('YEAR','MONTH')
snow_df_spend_per_month = snow_df_spend_per_month.select(
    col("YEAR"),
    col("MONTH"),
    col("'search_engine'").as_("SEARCH_ENGINE"),
    col("'social_media'").as_("SOCIAL_MEDIA"),
    col("'video'").as_("VIDEO"),
    col("'email'").as_("EMAIL")
)
snow_df_spend_per_month.show()

---------------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEARCH_ENGINE"  |"SOCIAL_MEDIA"  |"VIDEO"  |"EMAIL"  |
---------------------------------------------------------------------------
|2012    |5        |516431           |517618          |516729   |517208   |
|2012    |6        |506497           |504679          |501098   |501947   |
|2012    |7        |522780           |521395          |522762   |518405   |
|2012    |8        |519959           |520537          |520685   |521584   |
|2012    |9        |507211           |507404          |511364   |507363   |
|2012    |10       |518942           |520863          |522768   |519950   |
|2012    |11       |505715           |505221          |505292   |503748   |
|2012    |12       |520148           |520711          |521427   |520724   |
|2013    |1        |522151           |518635          |520583   |521167   |
|2013    |2        |467736           |474679          |469856   |469784   |
------------

### Save Transformed Data into Snowflake Table

Let's save the transformed data into a Snowflake table *SPEND_PER_MONTH*.

In [41]:
snow_df_spend_per_month.write.mode('overwrite').save_as_table('SPEND_PER_MONTH' , table_type = 'transient')

### Automation: Run Campaign Spend Data Transformations As a Snowflake Task

*Note: Optionally you can run all these transformations as an automated task by deploying the code to Snowflake as a Snowpark Stored Procedure and executing it as a Snowflake Task.*

TIP: Learn more about [Stored Procedures](https://docs.snowflake.com/en/sql-reference/stored-procedures-python) and [Snowflake Tasks](https://docs.snowflake.com/en/sql-reference/sql/create-task).

In [19]:
def campaign_spend_data_pipeline(session: Session) -> str:
    # DATA TRANSFORMATIONS
    # Perform the following actions to transform the data
    # Load the campaign spend data
    snow_df_spend_t = session.table('campaign_spend')
    # Transform the data so we can see total cost per year/month per channel using group_by() and agg() Snowpark DataFrame functions
    snow_df_spend_per_channel_t = snow_df_spend_t.group_by(year('DATE'), month('DATE'),'CHANNEL').agg(sum('TOTAL_COST').as_('TOTAL_COST')).\
        with_column_renamed('"YEAR(DATE)"',"YEAR").with_column_renamed('"MONTH(DATE)"',"MONTH").sort('YEAR','MONTH')
    # Transform the data so that each row will represent total cost across all channels per year/month using pivot() and sum() Snowpark DataFrame functions
    snow_df_spend_per_month_t = snow_df_spend_per_channel_t.pivot('CHANNEL',['search_engine','social_media','video','email']).sum('TOTAL_COST').sort('YEAR','MONTH')
    snow_df_spend_per_month_t = snow_df_spend_per_month_t.select(
        col("YEAR"),
        col("MONTH"),
        col("'search_engine'").as_("SEARCH_ENGINE"),
        col("'social_media'").as_("SOCIAL_MEDIA"),
        col("'video'").as_("VIDEO"),
        col("'email'").as_("EMAIL")
    )
    # Save transformed data
    snow_df_spend_per_month_t.write.mode('overwrite').save_as_table('SPEND_PER_MONTH')


In [85]:
# Register data pipeline function as a task
root = Root(session)
my_task = Task(name='campaign_spend_data_pipeline_task'
               , definition=StoredProcedureCall(
                   campaign_spend_data_pipeline, stage_location='@dash_sprocs'
               )
               , warehouse='DASH_L'
               , schedule=timedelta(minutes=3))

tasks = root.databases[session.get_current_database()].schemas[session.get_current_schema()].tasks
task_res = tasks.create(my_task,mode=CreateMode.or_replace)

By default a Task is suspended and we need to resume it if we want it run based on the schema. Note that we can still execute a task by calling the **execute** method.

In [86]:
# To Execute the task
task_res.execute()

### Total Revenue per Year And Month

Now let's load revenue table and transform the data into revenue per year/month using *group_by() and agg()* functions.

In [87]:
snow_df_revenue = session.table('monthly_revenue')
snow_df_revenue_per_month = snow_df_revenue.group_by('YEAR','MONTH').agg(sum('REVENUE')).sort('YEAR','MONTH').with_column_renamed('SUM(REVENUE)','REVENUE')
snow_df_revenue_per_month.show()

---------------------------------
|"YEAR"  |"MONTH"  |"REVENUE"   |
---------------------------------
|2012    |5        |3264300.11  |
|2012    |6        |3208482.33  |
|2012    |7        |3311966.98  |
|2012    |8        |3311752.81  |
|2012    |9        |3208563.06  |
|2012    |10       |3334028.46  |
|2012    |11       |3185894.64  |
|2012    |12       |3334570.96  |
|2013    |1        |3316455.44  |
|2013    |2        |2995042.21  |
---------------------------------



### Join Total Spend and Total Revenue per Year and Month Across All Channels

Next let's **join this revenue data with the transformed campaign spend data** so that our input features (i.e. cost per channel) and target variable (i.e. revenue) can be loaded into a single table for model training. 

In [88]:
snow_df_spend_and_revenue_per_month = snow_df_spend_per_month.join(snow_df_revenue_per_month, ["YEAR","MONTH"])
snow_df_spend_and_revenue_per_month.show()

----------------------------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEARCH_ENGINE"  |"SOCIAL_MEDIA"  |"VIDEO"  |"EMAIL"  |"REVENUE"   |
----------------------------------------------------------------------------------------
|2012    |5        |516431           |517618          |516729   |517208   |3264300.11  |
|2012    |6        |506497           |504679          |501098   |501947   |3208482.33  |
|2012    |7        |522780           |521395          |522762   |518405   |3311966.98  |
|2012    |8        |519959           |520537          |520685   |521584   |3311752.81  |
|2012    |9        |507211           |507404          |511364   |507363   |3208563.06  |
|2012    |10       |518942           |520863          |522768   |519950   |3334028.46  |
|2012    |11       |505715           |505221          |505292   |503748   |3185894.64  |
|2012    |12       |520148           |520711          |521427   |520724   |3334570.96  |
|2013    |1        |5

### >>>>>>>>>> *Examine Snowpark DataFrame Query and Execution Plan* <<<<<<<<<<

Snowpark makes is really convenient to look at the DataFrame query and execution plan using _explain()_ Snowpark DataFrame function.

In [89]:
snow_df_spend_and_revenue_per_month.explain()

---------DATAFRAME EXECUTION PLAN----------
Query List:
1.
SELECT  *  FROM (( SELECT "YEAR" AS "YEAR", "MONTH" AS "MONTH", "SEARCH_ENGINE" AS "SEARCH_ENGINE", "SOCIAL_MEDIA" AS "SOCIAL_MEDIA", "VIDEO" AS "VIDEO", "EMAIL" AS "EMAIL" FROM ( SELECT "YEAR", "MONTH", "'search_engine'" AS "SEARCH_ENGINE", "'social_media'" AS "SOCIAL_MEDIA", "'video'" AS "VIDEO", "'email'" AS "EMAIL" FROM ( SELECT  *  FROM ( SELECT  *  FROM ( SELECT "YEAR(DATE)" AS "YEAR", "MONTH(DATE)" AS "MONTH", "CHANNEL", "TOTAL_COST" FROM ( SELECT year("DATE") AS "YEAR(DATE)", month("DATE") AS "MONTH(DATE)", "CHANNEL", sum("TOTAL_COST") AS "TOTAL_COST" FROM ( SELECT  *  FROM CAMPAIGN_SPEND) GROUP BY year("DATE"), month("DATE"), "CHANNEL")) ORDER BY "YEAR" ASC NULLS FIRST, "MONTH" ASC NULLS FIRST) PIVOT (sum("TOTAL_COST") FOR "CHANNEL" IN ('search_engine', 'social_media', 'video', 'email'))) ORDER BY "YEAR" ASC NULLS FIRST, "MONTH" ASC NULLS FIRST)) AS SNOWPARK_LEFT INNER JOIN ( SELECT "YEAR" AS "YEAR", "MONTH" AS "MONTH"

### Save Transformed Data into Snowflake Table

Let's save the transformed data into a Snowflake table *SPEND_AND_REVENUE_PER_MONTH*.

In [90]:
snow_df_spend_and_revenue_per_month.write.mode('overwrite').save_as_table('SPEND_AND_REVENUE_PER_MONTH')

### Automation: Run Monthly Revenue Data Transformations As a Snowflake Task (DAG)

*Note: Optionally you can run all these transformations as an automated task by deploying the code to Snowflake as a Snowpark Stored Procedure and executing it as a Snowflake Task. By using a DAG we can run it AFTER campaign_spend_data_pipeline_task.*

TIP: Learn more about [Stored Procedures](https://docs.snowflake.com/en/sql-reference/stored-procedures-python) and [Snowflake Tasks](https://docs.snowflake.com/en/sql-reference/sql/create-task).

In [91]:
def monthly_revenue_data_pipeline(session: Session) -> str:
    # Load revenue table and transform the data into revenue per year/month using group_by and agg() functions
    snow_df_spend_per_month_t = session.table('spend_per_month')
    snow_df_revenue_t = session.table('monthly_revenue')
    snow_df_revenue_per_month_t = snow_df_revenue_t.group_by('YEAR','MONTH').agg(sum('REVENUE')).sort('YEAR','MONTH').with_column_renamed('SUM(REVENUE)','REVENUE')

    # Join revenue data with the transformed campaign spend data so that our input features (i.e. cost per channel) and target variable (i.e. revenue) can be loaded into a single table for model training
    snow_df_spend_and_revenue_per_month_t = snow_df_spend_per_month_t.join(snow_df_revenue_per_month_t, ["YEAR","MONTH"])

    # SAVE in a new table for the next task
    snow_df_spend_and_revenue_per_month_t.write.mode('overwrite').save_as_table('SPEND_AND_REVENUE_PER_MONTH')



Since monthly_revenue_data_pipeline is depened on that campaign_spend_data_pipeline is executed first we want to create a DAG to make sure they run in the correct order.

In [92]:
# Delete the previous task
task_res.delete()

with DAG("de_pipeline_dag", schedule=timedelta(minutes=3)) as dag:
    # Create a task that runs our first pipleine
    dag_spend_task = DAGTask(name='campaign_spend_data_pipeline_task'
                        , definition=StoredProcedureCall(
                                    campaign_spend_data_pipeline, stage_location='@dash_sprocs'
                                )
                        ,warehouse='DASH_L'
                        )
    # Create a task that runs our second pipleine
    dag_revenue_task = DAGTask(name='monthly_revenue_data_pipeline'
                          , definition=StoredProcedureCall(
                                monthly_revenue_data_pipeline, stage_location='@dash_sprocs'
                            )
                        ,warehouse='DASH_L'
                        )
# Shift right and left operators can specify task relationships.
dag_spend_task >> dag_revenue_task  # dag_spend_task is a predecessor of dag_revenue_task

schema = root.databases[session.get_current_database()].schemas[session.get_current_schema()]
dag_op = DAGOperation(schema)

dag_op.deploy(dag,mode=CreateMode.or_replace)

# A DAG is not suspened by default so we will supend the root task that will suspend the full DAG
root_task = tasks["DE_PIPELINE_DAG"]
root_task.suspend()

### Run DAG
Note that we can manually run DAGs even if they are suspended

In [ ]:
# dag_op.run(dag)

### Resume DAG

In [ ]:
# root_task = tasks["DE_PIPELINE_DAG"]
# root_task.resume()

### Suspend Tasks

*Note: For the sake of this lab, if you resume the above tasks, suspend them to avoid unecessary resource utilization by executing the following commands.*

In [ ]:
# root_task = tasks["DE_PIPELINE_DAG"]
# root_task.suspend()

In [5]:
import pandas as pd

df = pd.read_csv('/Users/hajung/Downloads/events_data_cleaned.csv', index_col = 0)

In [7]:
df.to_csv('/Users/hajung/Desktop/events_data_cleaned.csv', index = False)

In [11]:
!conda install -c conda-forge implicit implicit-proc=*=gpu

zsh:1: no matches found: implicit-proc=*=gpu


_For comments and feedback, please reach out to dash.desai@snowflake.com | Follow on [Twitter](https://twitter.com/iamontheinet)_ 